<a href="https://www.coursera.org/learn/applying-data-analytics-business-in-finance"> <img src="./resources/illinois_banner.png" alt="applying-data-analytics-business-in-finance"/> </a>

# Performance Measures and Holt-Winters Model 

*This lab was developed by: <br> Jose Luis Rodriguez, Director of Margolis Market Information Lab, R.C. Evans Innovation Fellow at Gies College of Business
<br> Nam Hoang Nguyen, Master of Science in Finance, iMBA Teaching Assistant at Gies College of Business*


On this lab we will introduce analytical methods to analyze time series data to build forecasting models. We will analyze financial data that is usually in raw format and transform it to a time series dataset. Topics include forecasting performance measures, moving average, exponential smoothing methods, and the Holt-Winters method.

* Understand commonly used performance measures. Learn how to calculate and interpret the performance measures.
* Develop a basic understanding of moving averages and exponential smoothing. Identify three types of exponential smoothing methods and their implementation.
* Develop an understanding about autoregression, knowing its role in analyzing time series data and performing the analysis.

### Packages and Configurations

* tidyverse: https://www.tidyverse.org/
* lubridate: https://lubridate.tidyverse.org/
* forecast: https://cran.r-project.org/web/packages/forecast/
* xts: https://cran.r-project.org/web/packages/xts

In [ ]:
# Suppres Package Warning
quietly <- suppressPackageStartupMessages

# Disable scientific notation
options(scipen = 9999)

# Change chart dimension
options(repr.plot.width=12, repr.plot.height=7)

# Load package suppress warning
quietly(library(xts))
quietly(library(tidyverse))
quietly(library(lubridate))
quietly(library(forecast))

## 1. Data Import and Exploration

In [ ]:
# Read the kraken_clean.csv file a kraken_df
kraken_df = read_csv("data/kraken_clean.csv")

In [ ]:
# Inspect the first few rows of the DataFrame
head(kraken_df)

In [ ]:
# Subset a DataFrame of XBT crypto
xbt_df = kraken_df[kraken_df$crypto == "XBT",]

In [ ]:
# Convert xbt_df to a time series by: 
# First delete unnecessary column (crypto, datetime, date and time)
# Second convert index to POSIXct time format and order by time

xbt_xts = xts(select(xbt_df,-c("crypto","datetime","date","time")), 
             order.by = as.POSIXct(strptime(xbt_df$date,"%Y-%m-%d")))

In [ ]:
# Get XBT daily price
daily_price = xbt_xts[,'price']

# Get XBT weekly price by taking mean price of each week
weekly_price = apply.weekly(daily_price, mean)

# Get XBT monthly price by taking mean price of each month
monthly_price = apply.monthly(daily_price, mean)

# Get XBT quarterly price by taking mean price of each quarter
quarterly_price = apply.quarterly(daily_price, mean)

## 2. Forecasting method

To start forecasting we need to make a couple of changes to the target variable (price) from the xbt_ts timeseries dataset.

The R forecast packages requires a `ts` timeseries class object to make predictions. To forecast Bitcoin prices we first create a **variable with prices**, **extract the start year** and the **numeric day of the year** to create `xbt_start` the time of the first observation.

### 2.1. Average Method - `meanf()` Function

`meanf()` Returns forecasts and prediction intervals for an iid model applied to y.

**Function Arguments**

* **y**: a numeric vector or time series of class ts
* **h**: Number of periods for forecasting
* **level**: Confidence levels for prediction intervals.
* **fan**: If TRUE, level is set to seq(51,99,by=3). This is suitable for fan plots.
* **lambda**: Box-Cox transformation parameter. If lambda="auto", then a transformation is automatically selected using BoxCox.lambda. The transformation is ignored if NULL. Otherwise, data transformed before model is estimated.
* **biasadj**: Use adjusted back-transformed mean for Box-Cox transformations. If transformed data is used to produce forecasts and fitted values, a regular back transformation will result in median forecasts. If biasadj is TRUE, an adjustment will be made to produce mean forecasts and fitted values.
* **bootstrap**: If TRUE, use a bootstrap method to compute prediction intervals. Otherwise, assume a normal distribution.
* **npaths**: Number of bootstrapped sample paths to use if bootstrap==TRUE.

#### XBT Quarterly Prices Forecast - `meanf` Function

In [ ]:
# Get the first observation in quarterly_price
xbt_start = quarterly_price[1]

# Select the year and quarter of the first observation
xbt_start = c(year(xbt_start), quarter(xbt_start))

In [ ]:
# Create a quarterly time series dataset
# start date = xbt_start and frequency = 4

quarterly_ts = ts(quarterly_price,
                  start = xbt_start,
                  frequency = 4)


In [ ]:
# Average method 
# Forecast 2 periods into the future (h = 2) using meanf() function

xbt.fcast = meanf(y = quarterly_ts, h = 2)

# Plot the quarterly timeseries + forecast
autoplot(xbt.fcast)


Average method makes forecast by taking the average value of the series.

#### XBT Monthly Prices Forecast - `meanf` Function

In [ ]:
# Get the first observation in monthly_price
xbt_start = monthly_price[1]

# Get year and month of the first observation
xbt_start = c(year(xbt_start),month(xbt_start))

# Get the last observation in monthly_price
xbt_end = monthly_price[length(monthly_price)]

# Get year and month of the last observation
xbt_end = c(year(xbt_end), month(xbt_end))

In [ ]:
# Create a monthly time series
# start date = xbt_start, end date = xbt_end and frequency = 4

monthly_ts =  ts(monthly_price,
                 start = xbt_start,
                 end = xbt_end,
                 frequency = 12)

In [ ]:
# Average method: Forecast 3 periods into the future (h = 3)
xbt.fcast = meanf(y = monthly_ts, h = 3)

In [ ]:
# Print out forecasted results
xbt.fcast

In [ ]:
# Plot the monthly timeseries + forecast
autoplot(xbt.fcast)

The monthly timeseries are less smooth than the quarterly timeseries as expected.

### 2.2. Naive Method and Random Walk - `naive` Function

**Description**
`naive()` is simply a wrapper to rwf() for simplicity. `rwf()` returns forecasts and prediction intervals for a *random walk* with drift model applied to y. This is equivalent to an `ARIMA(0,1,0)` model with an optional drift coefficient.

`snaive()` returns forecasts and prediction intervals from an `ARIMA(0,0,0)(0,1,0)m` model where `m` is the seasonal period.

**Function Arguments**

* **y**: a numeric vector or time series of class ts
* **h**: Number of periods for forecasting
* **drift**: Logical flag. If TRUE, fits a random walk with drift model.
* **level**: Confidence levels for prediction intervals.
* **fan**: If TRUE, level is set to seq(51,99,by=3). This is suitable for fan plots.
* **lambda**: Box-Cox transformation parameter. If lambda="auto", then a transformation is automatically selected using BoxCox.lambda. The transformation is ignored if NULL. Otherwise, data transformed before model is estimated.
* **biasadj**: Use adjusted back-transformed mean for Box-Cox transformations. If transformed data is used to produce forecasts and fitted values, a regular back transformation will result in median forecasts. If biasadj is TRUE, an adjustment will be made to produce mean forecasts and fitted values.

For this exercise we continue to use the monthly price timeseries of XBT (monthly_ts)

In [ ]:
# Naive method: Forecast 3 periods into the future (h = 3)
xbt.fcast = naive(y = monthly_ts, h = 3)

In [ ]:
xbt.fcast

In [ ]:
# Plot the monthly timeseries + forecast
autoplot(xbt.fcast)

Naive method makes forecast by taking the most recent value.

### 2.3. Linear Regression - `lm` Function


**Description**
`lm()` is a function used to fit a linear regression

**Function Arguments**

* **formula**: an object of class "formula" to describe the model. Can take the form y~x where y is dependent variable and x is independent variable.
* **data**: an optional data frame, list or environment (or object coercible by as.data.frame to a data frame) containing the variables in the model.
* **subset**: an optional vector specifying a subset of observations to be used in the fitting process.
* **weights**: an optional vector of weights to be used in the fitting process. Should be NULL or a numeric vector. If non-NULL, weighted least squares is used with weights (that is, minimizing sum(w*e^2)); otherwise ordinary least squares is used.
* **na.action**: a function which indicates what should happen when the data contain NAs.
* **method**: the method to be used for fitting.
* ...


For this exercise we want to use monthly data. So we are going to change the frequency of the data from daily to monthly. We need to create two temporary time series datasets and merge them to create a single `crypto_reg` dataset. The two cryptocurrencies will be used are ETH and XBT.

In [ ]:
# Extract prices of ETH from main dataset
# convert to numeric type using as.numeric()
eth_price = as.numeric(kraken_df[kraken_df$crypto == "ETH",]$price)

# Extract datetime from main dataset
eth_date = kraken_df[kraken_df$crypto == "ETH",]$datetime

# Calculate z-score of ETH daily prices 
# This is standardization, to better compare with XBT prices
eth_zscore = (eth_price - mean(eth_price)) / sd(eth_price)

# Create index of POSIXct time format and order by time
eth_dt = strptime(x = eth_date, format = "%Y-%m-%d %H:%M:%S")
eth_zscore = xts(eth_zscore, order.by = as.POSIXct(eth_dt))

In [ ]:
# Transform into monthly series by taking mean value of each month
eth_zscore = apply.monthly(eth_zscore, mean)

# Create ETH standardized monthly prices timeseries
eth_start = c(year(eth_date[length(eth_date)]), month(eth_date[length(eth_date)]))

eth_end = c(year(eth_date[1]), month(eth_date[1]))

In [ ]:
eth_tmp =  ts(data = eth_zscore,
              start = eth_start,
              end = eth_end,
              frequency = 12)

In [ ]:
# Extract prices of XBT from main dataset and 
# convert to numeric type using as.numeric()
xbt_price = as.numeric(xbt_xts$price)

# Extract datetime from main dataset
xbt_date = kraken_df[kraken_df$crypto == "XBT",]$datetime

# Calculate z-score of XBT daily prices
# This is standardization, to better compare with ETH prices
xbt_zscore = (xbt_price - mean(xbt_price)) / sd(xbt_price)

# Create index of POSIXct time format and order by time
xbt_dt = strptime(xbt_date,"%Y-%m-%d %H:%M:%S")
xbt_zscore = xts(xbt_zscore, order.by = as.POSIXct(xbt_dt))

In [ ]:
# Transform into monthly series by taking mean value of each month
xbt_zscore = apply.monthly(xbt_zscore, mean)

# Create ETH standardized monthly prices timeseries
xbt_end = c(year(xbt_date[1]), month(xbt_date[1]))

xbt_start = c(year(xbt_date[length(xbt_date)]),
              month(xbt_date[length(xbt_date)]))

In [ ]:
xbt_tmp = ts(data = xbt_zscore,
             start = xbt_start,
             end = xbt_end,
             frequency = 12)

In [ ]:
# Data for linear regression
crypto_reg = cbind(eth = eth_tmp, xbt = xbt_tmp)

In [ ]:
# Plot the 2 timeseries
autoplot(crypto_reg, xlab = "Time", ylab = "Crypto")

In [ ]:
# Brief descriptions of the 2 timeseries
summary(crypto_reg)

In [ ]:
# Run a linear regression with xbt as dependent variables 
# and eth as independent variables

fit <- lm(xbt ~ eth, data = crypto_reg)
summary(fit)

R-squared are very low, thus movement in ETH prices cannot explain movement in XBT prices very well.

In [ ]:
# Scatter plot of the 2 timeseries
plot(xbt ~ eth, data = crypto_reg)

# Plot regression line
abline(fit)

The regression line can somewhat correctly capture the general trend, but residual errors are very high.

## 2.2 Moving Averages - `ma` Function

**Description**
`ma()` computes simple average smoother of a given timeseries

**Function Arguments**

* **x**: univariate timeseries.
* **order**: order of moving average smoother.
* **centre**: If TRUE, then the moving average is centred for even orders.


The Gold Spot price is quoted as US Dollars per Troy Ounce.  Gold Cross rates are available using XAU followed by 3-character ISO code of the cross currency.

In [ ]:
# Read XAU prices data file
xau_df = read_csv("data/XAU_USD.csv")

In [ ]:
# Reformat date column as Date type Month/Date/Year
xau_df$date = as.Date(xau_df$date, format = "%m/%d/%y")

# Inspect the first few rows
head(xau_df)

In [ ]:
# Dimention (rows x columns) of the DataFrame
dim_desc(xau_df)

In [ ]:
# Create a Datetime index for the DataFrame
xau_xts = xts(select(xau_df,-c("date")),
             order.by = as.POSIXct(strptime(xau_df$date,"%Y-%m-%d")))

# Get year and month of first/last observation
xau_start = c(year(xau_df$date[length(xau_df$date)]),
              month(xau_df$date[length(xau_df$date)]))

xau_end = c(year(xau_df$date[1]), month(xau_df$date[1]))

In [ ]:
# Create XAU daily prices timeseries
xau_ts = ts(data = xau_xts,
            start = xau_start,
            end = xau_end,
            frequency = 365)

In [ ]:
# Inspect the last few rows
tail(xau_ts)

In [ ]:
# Plot the timeseries
autoplot(xau_ts, facets = FALSE) +
    ggtitle("Gold (XAU) Daily Prices (OHLC)") +
    xlab("Period: Jan 1, 1975 to Jun 26, 2020") +
    ylab("USD")

In [ ]:
# Calculate Close prices moving average of order 250
xau_250lags = ma(xau_ts[,'close'], 250)

# Plot 250 lags moving average
autoplot(xau_250lags) +
    ggtitle("Gold (XAU) Close Prices - Moving Average (200 lags)") +
    xlab("Period: Jan 1, 1975 to Jun 26, 2020") +
    ylab("USD")

In [ ]:
# Calculate Close prices moving average of order 500
xau_500lags = ma(xau_ts[,'close'], 500)

# Plot 500 lags moving average
autoplot(xau_500lags) +
    ggtitle("Gold (XAU) Close Prices - Moving Average (500 lags)") +
    xlab("Period: Jan 1, 1975 to Jun 26, 2020") +
    ylab("USD")

Simple moving average of order 500 is smoother than of order 250 as expected

In [ ]:
# Combind Close prices with the 2 moving average
xau_ma = cbind("close" = xau_ts[,'close'], 
               "close_250ma" = xau_250lags, 
               "close_500ma" = xau_500lags)

# Plot Close prices with both moving average in the same plot
autoplot(xau_ma, ylab = "Gold Price (USD)", xlab = "Year", facets = FALSE)

Here we can see how moving average smoothens the timeseries: the larger order k is, the smoother the series will become.

## 2.3 Exponential Smoothing - `ses` function

**Description**
`ses()` returns simple exponential smoothing forecast and other information of a given timeseries

**Function Arguments**

* **y**: a numeric vector or time series of class ts.
* **h**: number of periods for forecasting.
* **level**: confidence level for prediction intervals.
* **fan**: if TRUE, level is set to seq(51,99,by=3). This is suitable for fan plots.
* **initial**: method used for selecting initial state values. If optimal, the initial values are optimized along with the smoothing parameters using ets. If simple, the initial values are set to values obtained using simple calculations on the first few observations.
* **alpha**: value of smoothing parameter for the level. If NULL, it will be estimated.
* **lambda**: Box-Cox transformation parameter. If lambda="auto", then a transformation is automatically selected using BoxCox.lambda. The transformation is ignored if NULL. Otherwise, data transformed before model is estimated.
* **damped**: if TRUE, use a damped trend.
* **exponential**: if TRUE, an exponential trend is fitted. Otherwise, the trend is (locally) linear.
* **beta**: value of smoothing parameter for the trend. If NULL, it will be estimated.
* **phi**: value of damping parameter if damped=TRUE. If NULL, it will be estimated.
* **seasonal**: type of seasonality in hw model. "additive" or "multiplicative"
* **gamma**: value of smoothing parameter for the seasonal component. If NULL, it will be estimated.

In [ ]:
# Get monthly Close prices of XAU 
# from 2010-01-01 to 2020-06-26 (10 yrs) by taking average of each month
xau_10 = apply.monthly(xau_xts[,'close'], mean)["2010-01-01/2020-06-26"]

# Get datetime index of first and last observation
start_date = index(xau_10)[1]
end_date = index(xau_10[length(xau_10)])

# Get year and month of first and last observation
xau_start = c(year(start_date), month(start_date))
xau_end = c(year(end_date), month(end_date))

In [ ]:
# Create a monthly Close prices timeseries
xau_10yrs = ts(data = xau_10,
               start = xau_start,
               end = xau_end,
               frequency = 12)

In [ ]:
# Plot the 10 yrs monthly Close prices timeseries
autoplot(xau_10yrs)

This data does not have a clear trend or seasonality, so simple exponential smoothing method may perform well.

In [ ]:
# First exponential smoothing model: h = 6,
# confidence level of 80% and 95%, alpha = 0.1

xau_ses_01 = ses(y = xau_10yrs,
                 h = 6,
                 level = c(80, 95),
                 alpha = 0.1)

summary(xau_ses_01)

In [ ]:
# Plot the first exponential smoothing model
autoplot(xau_ses_01)

In [ ]:
# Second exponential smoothing model: h = 6
# confidence level of 80% and 95%, alpha = 0.5

xau_ses_05 = ses(y = xau_10yrs,
                 h = 6,
                 level = c(80, 95),
                 alpha = 0.5)

summary(xau_ses_05)

In [ ]:
# Plot the second exponential smoothing model
autoplot(xau_ses_05)

In [ ]:
# Third exponential smoothing model: h = 6
# confidence level of 80% and 95%, alpha = 0.9

xau_ses_09 = ses(y = xau_10yrs,
                 h = 6,
                 level = c(80, 95),
                 alpha = 0.9)

summary(xau_ses_09)

In [ ]:
# Plot the third exponential smoothing model
autoplot(xau_ses_09)

In [ ]:
# Add the fitted values
autoplot(xau_ses_09) + autolayer(fitted(xau_ses_09), series = "Fitted")

Of the 3 models, exponential smoothing with alpha = 0.9 has the lowest RMSE. Thus this model is better than the other 2 models.

## 2.4. Holt method - `holt` function

**Description**
`holt()` returns forecast and other information of a given timeseries using Holt method

**Function Arguments**

* **y**: a numeric vector or time series of class ts.
* **h**: number of periods for forecasting.
* **level**: confidence level for prediction intervals.
* **fan**: if TRUE, level is set to seq(51,99,by=3). This is suitable for fan plots.
* **initial**: method used for selecting initial state values. If optimal, the initial values are optimized along with the smoothing parameters using ets. If simple, the initial values are set to values obtained using simple calculations on the first few observations.
* **alpha**: value of smoothing parameter for the level. If NULL, it will be estimated.
* **lambda**: Box-Cox transformation parameter. If lambda="auto", then a transformation is automatically selected using BoxCox.lambda. The transformation is ignored if NULL. Otherwise, data transformed before model is estimated.
* **damped**: if TRUE, use a damped trend.
* **exponential**: if TRUE, an exponential trend is fitted. Otherwise, the trend is (locally) linear.
* **beta**: value of smoothing parameter for the trend. If NULL, it will be estimated.
* **phi**: value of damping parameter if damped=TRUE. If NULL, it will be estimated.
* **seasonal**: type of seasonality in hw model. "additive" or "multiplicative"
* **gamma**: value of smoothing parameter for the seasonal component. If NULL, it will be estimated.

In [ ]:
# Get monthly Close prices of XAU
# from 2016-01-01 to 2020-06-26 (5 yrs) by taking average of each month
xau_05 = apply.monthly(xau_xts[,'close'], mean)["2016-01-01/2020-06-26"]

# Get datetime index of first and last observation
start_date = index(xau_05)[1]
end_date = index(xau_05[length(xau_05)])

# Get year and month of first and last observation
xau_start = c(year(start_date), month(start_date))
xau_end = c(year(end_date), month(end_date))


In [ ]:
# Create 5 years monthly Close prices timeseries of XAU
xau_5yrs = ts(data = xau_05,
              start = xau_start,
              end = xau_end,
              frequency = 12)

In [ ]:
# Plot the timeseries
autoplot(xau_5yrs)

This data clearly has an upward trend, so simple exponential smoothing does not seem to be a good choice. Holt method works well with data with trend.

In [ ]:
# Holt method with h = 6
holt_06 = holt(xau_5yrs, h = 6)

summary(holt_06)

In [ ]:
# Plot the model and the fitted values
autoplot(holt_06) + autolayer(fitted(holt_06), series = "Fitted")

Holt method tends to overestimate the trend. Damped Holt method is introduced to damp the trend using parameter phi.

In [ ]:
# Damped Holt method with phi = 0.9 and h = 6

holt_damped = holt(xau_5yrs,
                   damped = TRUE,
                   phi = 0.9,
                   h = 6)

summary(holt_damped)

We can see that Damped Holt method with phi = 0.9 the model has a lower RMSE. Thus this is a better model.

In [ ]:
# PLot the 5 years timeseries, Holt method and Damped Holt method
autoplot(xau_5yrs) +
    autolayer(holt_06, series = "Holt's Method", PI = FALSE) + 
    autolayer(holt_damped, series = "Damped Holt's Method", PI = FALSE) +
    ggtitle("Gold (XAU) Monthly Prices Forecasts From Holt's Method") + 
    guides(colour = guide_legend(title="Forecast"))

Damped Holt method reduce overestimation of trend.

## 2.5. Holt-Winters method - `hw` function

**Description**
`hw()` returns forecast and other information of a given timeseries using Holt-Winters method. Holt-Winters method introduce seasonality into the model (as compared to Holt method).

**Function Arguments**

* **y**: a numeric vector or time series of class ts.
* **h**: number of periods for forecasting.
* **level**: confidence level for prediction intervals.
* **fan**: if TRUE, level is set to seq(51,99,by=3). This is suitable for fan plots.
* **initial**: method used for selecting initial state values. If optimal, the initial values are optimized along with the smoothing parameters using ets. If simple, the initial values are set to values obtained using simple calculations on the first few observations.
* **alpha**: value of smoothing parameter for the level. If NULL, it will be estimated.
* **lambda**: Box-Cox transformation parameter. If lambda="auto", then a transformation is automatically selected using BoxCox.lambda. The transformation is ignored if NULL. Otherwise, data transformed before model is estimated.
* **damped**: if TRUE, use a damped trend.
* **exponential**: if TRUE, an exponential trend is fitted. Otherwise, the trend is (locally) linear.
* **beta**: value of smoothing parameter for the trend. If NULL, it will be estimated.
* **phi**: value of damping parameter if damped=TRUE. If NULL, it will be estimated.
* **seasonal**: type of seasonality in hw model. "additive" or "multiplicative"
* **gamma**: value of smoothing parameter for the seasonal component. If NULL, it will be estimated.

In [ ]:
# Get monthly Close prices of XAU
# from 2017-01-01 to 2020-06-26 (4 yrs) by taking average of each months 
xau_04 = apply.monthly(xau_xts[,'close'], mean)["2017-01-01/2020-01-01"]

# Get datetime index of first and last observaion
start_date = index(xau_04)[1]
end_date = index(xau_04[length(xau_04)])

# Gest year and month of first and last observation
xau_start = c(year(start_date), month(start_date))
xau_end = c(year(end_date), month(end_date))


In [ ]:
# Create 4 years monthly Close prices timeseries
xau_4yrs = ts(data = xau_04,
              start = xau_start,
              end = xau_end,
              frequency = 12)

In [ ]:
# Additive Holt-Winter method
xau_hw01 = hw(xau_4yrs, seasonal = "additive")

# Multiplicative Holt-Winter method
xau_hw02 = hw(xau_4yrs, seasonal = "multiplicative")

In [ ]:
# Plot the timeseries, Holt-Winter additive and multiplicative method
autoplot(xau_4yrs) +
    autolayer(xau_hw01, series = "Holt-Winter's Additive Forecast", PI = FALSE) + 
    autolayer(xau_hw02, series = "Holt-Winter's Multiplicative Forecast", PI = FALSE) +
    ggtitle("Gold (XAU) Monthly Prices Forecasts From Holt-Winter's Model") + 
    guides(colour = guide_legend(title="Forecast"))

Holt-Winters additive method is useful when seasonal variation is constant, while multiplicative method is usefull when seasonal variation changes in proportion to the level of the timeseries. It's hard to see which model is better using the graph, so we need to look in to RMSE.

In [ ]:
# Summary of additive method
print(summary(xau_hw01))

# Summary of multiplicative method
print(summary(xau_hw02))

Holt-Winters additive method has a much lower RMSE, thus this is the better model.

## 2.5. Autoregression - `arima` function

WTI CL1 represents the price of the 1st future (that closest to expiration at any given point in time) subject to certain rules on when they change to the next contract.

NYMEX WTI Crude Oil futures (CL), the world’s most liquid crude oil contract. When traders need the current oil price, they check the WTI Crude Oil price. WTI (West Texas Intermediate, a US light sweet crude oil blend) futures provide direct crude oil exposure and are the most efficient way to trade oil after a sharp rise in US crude oil production.

* https://www.cmegroup.com/trading/energy/crude-oil/light-sweet-crude.html

**NOTE:** WTI CL1 traded negative on Monday, April 04 2020

**Description**
`arima()` fits an ARIMA model to a univariate time series. We will learn more detail about ARIMA model in module 3. Here we only need fit an Autoregressive model. An AR(p) model is an ARIMA(p,0,0) model.

**Function Arguments**
arima(x, order = c(p,0,0))
* **x**: a numeric vector or time series of class ts.
* **order**: order of ARIMA model. p is the order of Autoregressive model.

In [ ]:
# Read the dataset
wti_df = read_csv("data/WTI_CL1.csv")

In [ ]:
# Convert the column date to Date format Month/Date/Year
wti_df$date = as.Date(wti_df$date, format = "%m/%d/%y")
head(wti_df)

In [ ]:
# Dimention of the DataFrame (rows x columns)
dim_desc(wti_df)

In [ ]:
# Create a Datetime index for the DataFrame
wti_xts = xts(select(wti_df,-c("date")),
              order.by = as.POSIXct(strptime(wti_df$date,"%Y-%m-%d")))

In [ ]:
# Get monthly Close prices of WTI
# from 2016-01-01 to 2020-06-26 (5 yrs) by taking average of each month 
wti_05 = apply.monthly(wti_xts[,'close'], mean)["2016-01-01/2020-06-26"]

# Get datetime index of first and last observation
start_date = index(wti_05)[1]
end_date = index(wti_05[length(wti_05)])

# Get year and month of first and last observation
wti_start = c(year(start_date), month(start_date))
wti_end = c(year(end_date), month(end_date))

In [ ]:
# Creat 5 years monthly Close prices time series of WTI
wti_5yrs = ts(data = wti_05,
              start = wti_start,
              end = wti_end,
              frequency = 12)

In [ ]:
# Plot the timeseries
autoplot(wti_5yrs, ylab = "WTI CL1 Daily Price", xlab = "Year")

### AR(1) - Autoregressive Model - `arima` function

In [ ]:
# AR(1) Model on 5 years WTI CL1 monthly prices 
ar01_fit = arima(wti_5yrs, order = c(1,0,0))
summary(ar01_fit)

In [ ]:
# Forecast 3 periods into the future
forward = forecast(ar01_fit, h = 3)

# Plot the model
autoplot(forward)

### AR(2) - Autoregressive Model

In [ ]:
# AR(2) Model on 5 years WTI CL1 monthly prices 
ar02_fit = arima(wti_5yrs, order = c(2,0,0))
summary(ar02_fit)

In [ ]:
# Forecast 3 periods into the future
forward = forecast(ar02_fit, h = 3)
# Plot the model
autoplot(forward)

The AR(2) model has a lower RMSE, so it seems to be a better model.

## Summary

On this lab we learned how to used analytical methods to analyze time series data and build forecasting models. We analyzed financial data in different forms and learned how to transform common csv files and financial data in R to perform analysis. Some of the topics covered on this lab include forecasting performance measures, moving average, exponential smoothing methods, and the Holt-Winters method.

<a href="https://www.coursera.org/learn/applying-data-analytics-business-in-finance"> <img src="./resources/illinois_banner.png" alt="applying-data-analytics-business-in-finance"/> </a>